In [42]:
from fastkml import kml
#from fastkml import geometry
import pandas as pd
import params_scrap_syride as params
import analyze_functions
import os

In [27]:
# Chemin vers votre fichier KML
kml_file_path = "/Users/Adrien/Documents/paramoteur/syride/analyze_traces/syride_traces/AdrienM/traces/1916552/1916552.kml"

In [28]:
# Ouverture du fichier KML et lecture du contenu
with open(kml_file_path, 'rb') as kml_file:
    kml_document = kml_file.read()
    k = kml.KML()
    k.from_string(kml_document)

    for feature0 in k.features():
        for feature in feature0.features():
            if isinstance(feature, kml.Placemark):
                geometry = feature.geometry
                if geometry:
                    coords = list(geometry.coords)
                    # print(f"Placemark Name: {feature.name}")
                    # if hasattr(geometry, 'coords'):
                    #     print(f"Geometry Type: Point")
                    #     print(f"Coordinates: {coords}\n")
                    # elif hasattr(geometry, 'exterior'):
                    #     print(f"Geometry Type: Polygon")
                    #     print(f"Coordinates: {list(geometry.exterior.coords)}\n")
                    # elif hasattr(geometry, 'points'):
                    #     print(f"Geometry Type: LineString")
                    #     coords0 = list(point.coords for point in geometry.points)
                    #     coords.append(coords0)
                    #     print(f"Coordinates: {coords}\n")
                    # else:
                    #     print(f"Unknown Geometry Type\n")
    df = pd.DataFrame(coords, columns=["longitude", "latitude", "altitude"])
    

In [29]:
df = pd.DataFrame(coords, columns=["longitude", "latitude", "altitude"])
df.head()

,longitude,latitude,altitude
0,0.216767,45.609617,141.0
1,0.216850,45.609617,141.0
2,0.216967,45.609617,141.0
3,0.217083,45.609617,143.0
4,0.217233,45.609617,144.0


In [65]:
time_in_sec = (59*60)+53
time_in_sec

3593

In [66]:
df.shape

(3300, 3)

In [69]:
df['diff_Altitude'] = df['altitude'].diff()
df.head()

,longitude,latitude,altitude,Diff_Altitude,diff_Altitude
0,0.216767,45.609617,141.0,NaN,NaN
1,0.216850,45.609617,141.0,0.0,0.0
2,0.216967,45.609617,141.0,0.0,0.0
3,0.217083,45.609617,143.0,2.0,2.0
4,0.217233,45.609617,144.0,1.0,1.0


In [71]:
df[df["diff_Altitude"]>4]

,longitude,latitude,altitude,Diff_Altitude,diff_Altitude
15,0.218633,45.609233,173.0,5.0,5.0
117,0.231100,45.606633,383.0,5.0,5.0
1795,0.323383,45.502283,615.0,5.0,5.0
1854,0.327483,45.496950,647.0,5.0,5.0
2188,0.351467,45.481317,520.0,5.0,5.0


In [46]:
def get_kml_files(directory: str)->dict:
    """
    Fonction qui itère l'ensemble des dossiers
    dans le dossier d'un pilote et qui recherhe tous les
    fichiers .kml

    arguments:
        directory: str = chemin du dossier dans lequel
        se trouvent les traces.
    
    return:
        dict de toutes les traces kml pour un pilote donné, avec :
            * key = le numéro de la trace
            * value = le chemin de la trace

    """
    kml_files = {}
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".kml") and file[:-5].isdigit():
                kml_files[file[:-4]] = os.path.join(root, file)
    return kml_files

def get_first_coordinates(kml_file_path:str) -> list:
    """
    Fonction permettant l'ouverture des fichiers kml et de l'extraction
    des premières coordonnées longitude/latitude.

    arguments:
        kml_file_path: str = chemin du dossier d'un fichier kml.
    
    return:
        liste des premières coordonnées longitude/latitude de la trace.

    """
    # Ouverture du fichier KML et lecture du contenu
    with open(kml_file_path, 'rb') as kml_file:
        kml_document = kml_file.read()
        k = kml.KML()
        k.from_string(kml_document)

        for feature0 in k.features():
            for feature in feature0.features():
                if isinstance(feature, kml.Placemark):
                    geometry = feature.geometry
                    if geometry:
                        coords = list(geometry.coords)
                    else:
                        coords = [(0, 0, 0)]
    
    longitude = coords[0][0]
    latitude = coords[0][1]

    return [longitude, latitude]

def create_dataframe_from_all_kml(directory:str)-> pd.DataFrame:
    """
    Fonction permettant de créer une dataframe avec l'ensemble des coordonnées
    longitude, latitude pour toutes les traces kml contenues dans le dossier d'un pilote.

    arguments:
        directory:str = le chemin du dossier contenant tous les dossiers / traces kml.

    
    return:
        df contenant pour chaque trace les longitudes, latitude du premier point.

    """
    dict_kml_files = get_kml_files(directory = directory)
    
    dict_first_coordinates = {flight: get_first_coordinates(kml_file_path) for flight, kml_file_path in dict_kml_files.items()}

    df = pd.DataFrame(dict_first_coordinates)
    df = df.transpose()
    df = df.reset_index(drop=False)
    df.columns = ['num_activite', 'longitude', 'latitude']

    return df

In [11]:
common_filename = params.main_path
dict_filenames = params.dict_pilotes

name = next(iter(dict_filenames.items()))[0]
foldername = next(iter(dict_filenames.items()))[1]

full_filename = f"{common_filename}{foldername}/"


In [35]:
dict_kml_files = get_kml_files(directory = full_filename)

In [47]:
df_long_lat = create_dataframe_from_all_kml(dict_kml_files=dict_kml_files)
df_long_lat.head()

,num_activite,longitude,latitude
0,1979467,-0.322683,46.153483
1,2079412,0.171283,45.559600
2,2035366,0.252467,45.486217
3,2228584,-0.865783,45.911267
4,2190658,-0.314150,46.157350


In [43]:
df_data = analyze_functions.create_dataframe_from_json(directory=full_filename)

In [48]:
df_full = pd.merge(df_data, df_long_lat, on='num_activite', how='left')
df_full.head()

,pilote,id_activite,num_activite,types,site,date,heure,flight_time,voile,distance,...,vitesse_max,vitesse_moyenne,plafond,gain,duree_vol,vario_max,g_max,adresse_zip,longitude,latitude
0,AdrienM,2069214,1979467,Vol moteur,Site Secondigne Sur Belle 01,16/02/2023,12h49,00:26:14,Daytona,11,...,70,25.2,479,419,00:26:14,3.4,1.5,https://www.syride.com/scripts/downloadZIP.php...,-0.322683,46.153483
1,AdrienM,2187786,2079412,Vol moteur,Fayard,13/05/2023,09h19,00:52:36,Daytona,27,...,61,30.8,751,615,00:52:36,2.9,1.6,https://www.syride.com/scripts/downloadZIP.php...,0.171283,45.559600
2,AdrienM,2139896,2035366,Vol moteur,Magnac Lavalette Treuil,09/04/2023,19h13,00:49:02,Flex-one,28,...,62,34.3,792,667,00:49:02,2.8,1.5,https://www.syride.com/scripts/downloadZIP.php...,0.252467,45.486217
3,AdrienM,2334386,2228584,Vol moteur,Cabariot,14/08/2023,20h56,00:29:53,Daytona,15,...,71,30.1,358,354,00:29:53,2.6,1.4,https://www.syride.com/scripts/downloadZIP.php...,-0.865783,45.911267
4,AdrienM,2293232,2190658,Vol moteur,Site Secondigne Sur Belle 01,22/07/2023,09h08,00:39:38,Daytona,22,...,67,33.3,542,497,00:39:38,3.1,1.2,https://www.syride.com/scripts/downloadZIP.php...,-0.314150,46.157350


In [57]:
def get_mean_site_coordinates(df:pd.DataFrame)->pd.DataFrame:
    """
    Fonction qui, à partir d'un dataframe contenant une colonne "site"
    et des colonnes longitude/latitude, calcule les longitudes/latitudes
    moyennes pour chaque sites.
    """

    df_long_lat_by_site = df[["site", "longitude", "latitude"]].groupby("site", as_index=False).mean()

    dict_mean_long_by_site = {site: longitude for site, longitude in zip(df_long_lat_by_site['site'], df_long_lat_by_site['longitude'])}
    dict_mean_lat_by_site = {site: latitude for site, latitude in zip(df_long_lat_by_site['site'], df_long_lat_by_site['latitude'])}

    df["mean_longitude"] = df["site"].map(dict_mean_long_by_site)
    df["mean_latitude"] = df["site"].map(dict_mean_lat_by_site)

    return df


,pilote,id_activite,num_activite,types,site,date,heure,flight_time,voile,distance,...,plafond,gain,duree_vol,vario_max,g_max,adresse_zip,longitude,latitude,mean_longitude,mean_latitude
0,AdrienM,2069214,1979467,Vol moteur,Site Secondigne Sur Belle 01,16/02/2023,12h49,00:26:14,Daytona,11,...,479,419,00:26:14,3.4,1.5,https://www.syride.com/scripts/downloadZIP.php...,-0.322683,46.153483,-0.320620,46.154358
1,AdrienM,2187786,2079412,Vol moteur,Fayard,13/05/2023,09h19,00:52:36,Daytona,27,...,751,615,00:52:36,2.9,1.6,https://www.syride.com/scripts/downloadZIP.php...,0.171283,45.559600,0.171318,45.559763
2,AdrienM,2139896,2035366,Vol moteur,Magnac Lavalette Treuil,09/04/2023,19h13,00:49:02,Flex-one,28,...,792,667,00:49:02,2.8,1.5,https://www.syride.com/scripts/downloadZIP.php...,0.252467,45.486217,0.252446,45.487330
3,AdrienM,2334386,2228584,Vol moteur,Cabariot,14/08/2023,20h56,00:29:53,Daytona,15,...,358,354,00:29:53,2.6,1.4,https://www.syride.com/scripts/downloadZIP.php...,-0.865783,45.911267,-0.865992,45.911304
4,AdrienM,2293232,2190658,Vol moteur,Site Secondigne Sur Belle 01,22/07/2023,09h08,00:39:38,Daytona,22,...,542,497,00:39:38,3.1,1.2,https://www.syride.com/scripts/downloadZIP.php...,-0.314150,46.157350,-0.320620,46.154358
